In [1]:
import numpy as np
import pandas as pd


In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data[train_data['target'] == 1].keyword.value_counts()

outbreak       39
derailment     39
wreckage       39
debris         37
oil%20spill    37
               ..
body%20bags     1
ruin            1
blazing         1
epicentre       1
electrocute     1
Name: keyword, Length: 220, dtype: int64

In [4]:
'''import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')
# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")
# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector
# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector'''

'import spacy\n# Load the spacy model that you have installed\nnlp = spacy.load(\'en_core_web_md\')\n# process a sentence using the model\ndoc = nlp("This is some text that I am processing with Spacy")\n# It\'s that simple - all of the vectors and words are assigned after this point\n# Get the vector for \'text\':\ndoc[3].vector\n# Get the mean vector for the entire sentence (useful for sentence classification etc.)\ndoc.vector'

In [5]:
caps_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
caps_letters = [char for char in caps_letters]

train_data[train_data.text.str.isupper()].target.value_counts()

0    37
1    13
Name: target, dtype: int64

In [9]:
train_data[train_data.text.str.contains(months, case=False)]

TypeError: unhashable type: 'list'

In [8]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'November', 'October', 'December']


In [9]:
train_data[train_data['target'] == 1]['text'].sample().values

array(['#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade be\x89Û_'],
      dtype=object)

In [10]:
train_data[train_data.text.str.contains('', case=False)].target.value_counts()


0    4342
1    3271
Name: target, dtype: int64

##### True Positive tweets tend to result in multiple tweets when searched in twitter

In [11]:
## Feature ideas
### All caps
### Ausrufezeichen
### Youtube
### Internet links
train_data[train_data.text.str.contains('http')].target.value_counts() # Mehr wahrscheinlichkeit von Disaster tweet
### @ 
train_data[train_data.text.str.contains('@', case = False)].target.value_counts() # Entries with @ tends to be less serious
### Text Containing news
train_data[train_data.text.str.contains('news', case = False)].target.value_counts() 
### I sentences
train_data[train_data.text.str.contains('i ', case=False)].target.value_counts()




0    1030
1     456
Name: target, dtype: int64

## Define keyword feature generator method

In [66]:
def keyword_feature_generator(keyword, feature_name, data = train_data):
    data[feature_name] = 0
    idx = data[data.text.str.contains(keyword, case=False)].index
    train_data.loc[idx, feature_name] = 1

## Add I sentence feature

In [12]:
keyword = 'i '
train_data[train_data.text.str.contains(keyword)].target.value_counts() 


0    238
1    216
Name: target, dtype: int64

In [13]:
keyword_feature_generator(keyword, 'i_included')

NameError: name 'keyword_feature_generator' is not defined

## Add http feature

In [49]:
train_data[train_data.text.str.contains('http')].target.value_counts() # Mehr wahrscheinlichkeit von Disaster tweet


1    2172
0    1799
Name: target, dtype: int64

In [50]:
train_data['http_included'] = 0
http_idx = train_data[train_data.text.str.contains('http', case=False)].index
train_data.loc[http_idx,'http_included'] = 1

## Add @ feature

In [51]:
train_data[train_data.text.str.contains('@')].target.value_counts() 

0    1363
1     676
Name: target, dtype: int64

In [52]:
train_data['at_included'] = 0
at_idx = train_data[train_data.text.str.contains('@', case=False)].index
train_data.loc[at_idx,'at_included'] = 1

## Add news feature

In [53]:
search_keyword = 'news'

In [54]:
train_data[train_data.text.str.contains(search_keyword, case = False)].target.value_counts() 


1    244
0     91
Name: target, dtype: int64

In [55]:
train_data[search_keyword + '_included'] = 0
news_idx = train_data[train_data.text.str.contains(search_keyword, case=False)].index
train_data.loc[news_idx, search_keyword+'_included'] = 1

In [63]:
train_data

,id,keyword,location,text,target,i_sentence,http_included,at_included,news_included,you_included
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,0,1,0,0,0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,0,0,1,0,0
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,0,1,0,0,0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,0,0,0,0,0
